In [2]:
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
from tokenizer import Tokenizer
from file_dataset import FileDataset
from transformer_model import build_transformer

In [3]:
tokenizer = Tokenizer()
VOCAB_SIZE = tokenizer.get_vocab_size()

In [4]:
dataset = FileDataset("/Users/daniilogorodnikov/dataset/app", 'sha256', 128)

100%|██████████| 780/780 [00:36<00:00, 21.46it/s]


In [15]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [16]:
transformer = build_transformer(vocab_size=VOCAB_SIZE,
                                d_model=512,
                                max_seq_len=128,
                                d_ff=1024,
                                dropout=0.1,
                                n_layers=6,
                                n_heads=8,
                                factor=2)

In [17]:
import torch.nn as nn

In [20]:
from tqdm import tqdm

In [18]:
criterion = nn.CrossEntropyLoss(ignore_index=5)
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-4)

In [23]:
for epoch in range(100):
    for batch in (pbar := tqdm(dataloader)):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']

        # Forward pass
        outputs = transformer(input_ids, attention_mask)

        # Расчет потерь (сравниваем выход с входом)
        loss = criterion(outputs.view(-1, outputs.shape[-1]), input_ids.view(-1))

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Loss: {loss.item():.4f}")

Loss: 11.7947:   0%|          | 120/815435 [00:42<80:10:54,  2.82it/s] 


KeyboardInterrupt: 

In [14]:
batch['attention_mask'].shape

torch.Size([10, 128])